# DD142X

### Prerequisites
* Python v3.7
* Tensorflow v1.4.2 
* Installera massa skit med pip för att få det att fungera allteftersom Python klagar.
* Jag ändrade i train_image_classifier.py för att träna på CPU.
* Typ 100 GB hårddiskutrymme.

Det kan funka med andra versioner, men inte Tensorflow 2.

### Ladda in directories

In [2]:
# Set pretrained checkpoint dir.
%env PRETRAINED_CHECKPOINT_DIR=/Users/johan/Dev/kexet/pretrained

# Where the training (fine-tuned) checkpoint and logs will be saved to.
%env TRAIN_DIR=/Users/johan/Dev/kexet/checkpoints

# Where the dataset is saved to.
%env DATASET_DIR=/Users/johan/Dev/kexet/data

env: PRETRAINED_CHECKPOINT_DIR=/Users/johan/Dev/kexet/pretrained
env: TRAIN_DIR=/Users/johan/Dev/kexet/checkpoints
env: DATASET_DIR=/Users/johan/Dev/kexet/data


### Ladda ner en förtränad inception v3

Kör bara första gången.

In [ ]:
### Download the model if you haven't yet.
!mkdir ${PRETRAINED_CHECKPOINT_DIR}
!wget http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz
!tar -xvf inception_v3_2016_08_28.tar.gz
!mv inception_v3.ckpt ${PRETRAINED_CHECKPOINT_DIR}
!rm inception_v3_2016_08_28.tar.gz

### Konvertera ISIC data

Kör första gången och lägg sen manuellt in i mappar.

In [ ]:
import json
import os
import shutil

path_to_descriptions = '/Users/johan/Dev/kexet/rawdata'

final_path = '/Users/johan/Dev/kexet/data/skincancer'

for root, dirs, files in os.walk(path_to_descriptions):
    files = [f for f in files if not f[0] == '.']
    files = [f for f in files if not f[-1] == 'g']
    dirs[:] = [d for d in dirs if not d[0] == '.']
    
    for name in files:
        src = root + '/' + name
        try:
            with open(src) as data_file:
                #print(name)
                jsonList = json.load(data_file)

                disease = jsonList['meta']['clinical']['diagnosis']
                b_or_m = jsonList['meta']['clinical']['benign_malignant']
                if b_or_m is None:
                    b_or_m = 'neither'
                image_src_path = path_to_descriptions + '/Images/' + name + '.jpeg'
                disease_dir = final_path + '/' + disease + '/' + b_or_m
                if not os.path.exists(disease_dir):
                    os.makedirs(disease_dir)

                data_file.close()
                shutil.copy(image_src_path, disease_dir)
        except:
            continue
            

            
print("Finished!")

In [ ]:
import download_and_convert_cancer as dcc
dcc.run("/Users/johan/Dev/kexet/data")

### Transfer learning
* Ladda vikterna från den generella färdigtränade modellen, men exkludera sista lagret med "checkpoint_exclude_scopes". 
* Träna om vikterna i detta lager med "trainable_scopes".

In [3]:
# Fine-tune only the new layers for 1000 steps.
!python3 train_image_classifier.py \
  --train_dir=${TRAIN_DIR} \
  --dataset_name=cancer \
  --dataset_split_name=train \
  --dataset_dir=${DATASET_DIR} \
  --model_name=inception_v3 \
  --checkpoint_path=${PRETRAINED_CHECKPOINT_DIR}/inception_v3.ckpt \
  --checkpoint_exclude_scopes=InceptionV3/Logits,InceptionV3/AuxLogits \
  --trainable_scopes=InceptionV3/Logits,InceptionV3/AuxLogits \
  --max_number_of_steps=100 \
  --batch_size=100 \
  --learning_rate=0.001 \
  --learning_rate_decay_type=fixed \
  --save_interval_secs=600 \
  --save_summaries_secs=600 \
  --log_every_n_steps=100 \
  --optimizer=rmsprop \
  --weight_decay=0.00004 \
  --opt_epsilon=0.1



W0510 16:38:44.781055 4720817600 module_wrapper.py:139] From train_image_classifier.py:417: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0510 16:38:44.781250 4720817600 module_wrapper.py:139] From train_image_classifier.py:417: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

Instructions for updating:
Please switch to tf.train.create_global_step
W0510 16:38:44.781970 4720817600 deprecation.py:323] From train_image_classifier.py:431: create_global_step (from tensorflow.contrib.framework.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.create_global_step

W0510 16:38:44.786154 4720817600 module_wrapper.py:139] From /Users/johan/Dev/kexet/models/research/slim/datasets/cancer.py:37: The name tf.FixedLenFeature is deprecated. Please use tf.io.FixedLenFeature instead.


W0510 16:38:44.786844 4720817600 module_w


W0510 16:38:44.982571 4720817600 module_wrapper.py:139] From train_image_classifier.py:504: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.


W0510 16:38:44.982797 4720817600 module_wrapper.py:139] From train_image_classifier.py:504: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.


W0510 16:38:44.983101 4720817600 module_wrapper.py:139] From /Users/johan/Dev/kexet/models/research/slim/deployment/model_deploy.py:192: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.


W0510 16:38:44.983249 4720817600 module_wrapper.py:139] From /Users/johan/Dev/kexet/models/research/slim/deployment/model_deploy.py:192: The name tf.get_variable_scope is deprecated. Please use tf.compat.v1.get_variable_scope instead.

Instructions for updating:
Please use `layer.__call__` method instead.
W0510 16:38:44.985006 4720817600 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorfl


W0510 16:38:49.439949 4720817600 module_wrapper.py:139] From train_image_classifier.py:382: The name tf.gfile.IsDirectory is deprecated. Please use tf.io.gfile.isdir instead.


W0510 16:38:49.441204 4720817600 module_wrapper.py:139] From train_image_classifier.py:387: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:Fine-tuning from /Users/johan/Dev/kexet/pretrained/inception_v3.ckpt
I0510 16:38:49.441358 4720817600 train_image_classifier.py:387] Fine-tuning from /Users/johan/Dev/kexet/pretrained/inception_v3.ckpt
Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
W0510 16:38:50.284988 4720817600 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow_core/contrib/slim/python/slim/learning.py:742: Supervisor.__init__ (from tensorflow.python.training.supervisor) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.MonitoredTrainingSe

In [4]:
# Run evaluation.
!python3 eval_image_classifier.py \
  --checkpoint_path=${TRAIN_DIR} \
  --eval_dir=${TRAIN_DIR} \
  --dataset_name=cancer \
  --dataset_split_name=validation \
  --dataset_dir=${DATASET_DIR} \
  --model_name=inception_v3



W0510 18:47:13.344049 4437976512 module_wrapper.py:139] From eval_image_classifier.py:97: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0510 18:47:13.344250 4437976512 module_wrapper.py:139] From eval_image_classifier.py:97: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

Instructions for updating:
Please switch to tf.train.get_or_create_global_step
W0510 18:47:13.344933 4437976512 deprecation.py:323] From eval_image_classifier.py:99: get_or_create_global_step (from tensorflow.contrib.framework.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.get_or_create_global_step

W0510 18:47:13.348997 4437976512 module_wrapper.py:139] From /Users/johan/Dev/kexet/models/research/slim/datasets/cancer.py:37: The name tf.FixedLenFeature is deprecated. Please use tf.io.FixedLenFeature instead.


W0510 18:47:13.349673 4437

Instructions for updating:
Please switch to tf.metrics.accuracy. Note that the order of the labels and predictions arguments has been switched.
W0510 18:47:15.862266 4437976512 deprecation.py:323] From eval_image_classifier.py:167: streaming_accuracy (from tensorflow.contrib.metrics.python.ops.metric_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.metrics.accuracy. Note that the order of the labels and predictions arguments has been switched.
Instructions for updating:
Please use `streaming_sparse_recall_at_k`, and reshape labels from [batch_size] to [batch_size, 1].
W0510 18:47:15.874540 4437976512 deprecation.py:323] From eval_image_classifier.py:169: streaming_recall_at_k (from tensorflow.contrib.metrics.python.ops.metric_ops) is deprecated and will be removed after 2016-11-08.
Instructions for updating:
Please use `streaming_sparse_recall_at_k`, and reshape labels from [batch_size] to [batch_size, 1].
Instructions for updat

INFO:tensorflow:Evaluation [16/22]
I0510 18:50:21.054650 4437976512 evaluation.py:167] Evaluation [16/22]
INFO:tensorflow:Evaluation [18/22]
I0510 18:50:40.100217 4437976512 evaluation.py:167] Evaluation [18/22]
INFO:tensorflow:Evaluation [20/22]
I0510 18:51:01.007409 4437976512 evaluation.py:167] Evaluation [20/22]
INFO:tensorflow:Evaluation [22/22]
I0510 18:51:20.576877 4437976512 evaluation.py:167] Evaluation [22/22]
eval/Accuracy[0.880909085]
eval/Recall_5[1]
INFO:tensorflow:Finished evaluation at 2020-05-10-18:51:21
I0510 18:51:21.463001 4437976512 evaluation.py:275] Finished evaluation at 2020-05-10-18:51:21
